In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
idx = ('Soil', 'Lake')

In [4]:
def do_optim(ds, ID, soil, depo, lake, printout=True) :

    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ExCa'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['ExMg'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ExNa'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['ExK'])
    
    
    params_a, comparisons_a = magicopt.get_acid_anion_setup(ds, idx, 'Observed')
    
    res_pars1, nfev1 = magicopt.calib(ds, params_a, comparisons_a, method='nelder')# method='leastsq')
    
    
    ds.set_parameter_double('Initial exchangeable Ca on soil as % of CEC', ['Soil'], soil['ECa-0'])
    ds.set_parameter_double('Initial exchangeable Mg on soil as % of CEC', ['Soil'], soil['EMg-0'])
    ds.set_parameter_double('Initial exchangeable Na on soil as % of CEC', ['Soil'], soil['ENa-0'])
    ds.set_parameter_double('Initial exchangeable K on soil as % of CEC', ['Soil'], soil['EK-0'])
    
    params, comparisons = magicopt.get_base_cations_setup(ds, 
                            idx, 'Observed', usemax=True, obsE=True)
    
    #params.pretty_print()
    #Set initial guess to the present-day value
    params['ECa_S'].value = soil['ExCa']+20
    params['EMg_S'].value = soil['ExMg']
    params['ENa_S'].value = soil['ExNa']
    params['EK_S'].value = soil['ExK']
    
    params['WCa_S'].value*=0.2
    params['WMg_S'].value*=0.2
    params['WNa_S'].value*=0.2
    params['WK_S'].value*=0.2
    
    params['WK_S'].min = max(0.0, soil['UptK'] - depo['K'])
    
    
    ntries = 0
    err=0
    abserr=0
    while (abserr>1 and ntries < 10) or ntries==0 :
        if err < 0. :
            params['WCa_S'].value*=2
            if ID==3096:
                params['ECa_S'].value+=5
        elif err > 0.:
            params['WCa_S'].value/=2
            if ID==3096:
                params['ECa_S'].value-=5
                
        res_pars2, nfev = magicopt.calib(ds, params, comparisons, 
                                    #method='nelder'
                                    method='leastsq'
                                    )
        
        sim = ds.get_result_series('Ca(2+) ionic concentration', ['Lake'])
        obs = ds.get_input_series('Observed Ca', [], alignwithresults=True)
        err = np.nansum(sim-obs)
        abserr = np.nansum(np.abs(sim-obs))
        
        ntries += 1
    
    #if np.abs(err)>1.:
    #    high_error_ids.append((ID,err))
    #    print('High Ca error (abs>1)')

    if printout :
        comp = comparisons
        comp += comparisons_a
        comp += [('SO4(2-) ionic concentration', ['Lake'], 'Observed SO4', [])]

        for comparison in comp :
            simname, simindexes, obsname, obsindexes, *_ = comparison

            sim = ds.get_result_series(simname, simindexes)
            obs = ds.get_input_series(obsname, obsindexes, alignwithresults=True)

            err = np.sqrt(np.nansum(np.square(sim-obs)))

            print('Error of %s: RSSE(sim-obs): %f' % (obsname, err))

        print('\n')
        res_pars1.pretty_print()
        res_pars2.pretty_print()
        print('\n')
        
        ds.write_parameters_to_file('MobiusFiles/params_%d.dat'%ID)
        ds.write_inputs_to_file('MobiusFiles/inputs_%d.dat'%ID)
    
    return res_pars1, res_pars2


def do_fuzzy(ds, ID, soil, depo, lake):
    
    nruns = 10
    
    fuzzy_params = [
        ('Observed NO3', [], 0.9, 1.1, 'obs'),
        ('Observed Ca', [], 0.9, 1.1, 'obs'),
        ('Observed Mg', [], 0.9, 1.1, 'obs'),
        ('Observed Na', [], 0.9, 1.1, 'obs'),
        ('Observed K', [], 0.9, 1.1, 'obs'),
        ('Depth', ['Soil'], 0.9, 1.1, 'par'),
    ]
    
    multipliers = np.zeros((len(fuzzy_params), nruns))
    values      = np.zeros((len(fuzzy_params), nruns))
    
    respars1 = []
    respars2 = []
    
    for idx, par in enumerate(fuzzy_params) :
        name, index, minimum, maximum, partype = par
        multipliers[idx, :] = np.random.uniform(minimum, maximum, nruns)
        
    for run in range(nruns) :
        ds2 = ds.copy()
        for idx, par in enumerate(fuzzy_params):
            name, index, minimum, maximum, partype = par
            if partype == 'obs':
                values = ds2.get_input_series(name, index)
                values *= multipliers[idx, run]
                ds2.set_input_series(name, index, values)
            elif partype == 'par':
                value = ds2.get_parameter_double(name, index)
                value *= multipliers[idx, run]
                ds2.set_parameter_double(name, index, value)
        res1,res2 = do_optim(ds2, ID, soil, depo, lake, printout=True)
        
        respars1.append(res1)
        respars2.append(res2)
        
        ds2.delete()
    
    #TODO: We need to store them somehow instead
    #return fuzzy_params, multipliers, respars1, respars2

In [ ]:
#soilfile, lakefile1, lakefile2, depofile, seqfile = excelfiles
excelfiles = ('CCE-soil BJC.xls', 'CCE-lake BJC.xls', 'WRI-lake 2019.xls', 'CCE-depo BJC.xls', 'CCE-seqs BJC.xls')

#multi.do_magic_loop(dataset, excelfile, do_fuzzy, do_id=9)
multi.do_magic_loop(dataset, excelfiles, do_optim, limit_num = 100)#, do_id=9)


#print('IDs with error in Ca > 1: ', high_error_ids)

Running lake Hokksjøen (ID 9)

Error of Observed Ca: RSSE(sim-obs): 0.314168
Error of Observed Mg: RSSE(sim-obs): 1.368018
Error of Observed Na: RSSE(sim-obs): 20.775500
Error of Observed K: RSSE(sim-obs): 0.729593
Error of Observed ECa: RSSE(sim-obs): 0.000000
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 6.792197
Error of Observed SO4: RSSE(sim-obs): 17.871229


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     26.15     0.01    88.54 4.464e-14     True     None     None
EK_S      5.992     0.01       15 1.062e-14     True     None     None
EMg_S     12.08     0.01    36.41 2.026e-14     True     None     None
ENa_S     5.601     0.01    13.08 1.097e-14     True     None     

Error of Observed Ca: RSSE(sim-obs): 20.314859
Error of Observed Mg: RSSE(sim-obs): 1.540589
Error of Observed Na: RSSE(sim-obs): 8.333337
Error of Observed K: RSSE(sim-obs): 0.011297
Error of Observed ECa: RSSE(sim-obs): 0.000000
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 0.149870
Error of Observed SO4: RSSE(sim-obs): 7.238160


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     18.27     0.01    84.58 5.108e-14     True     None     None
EK_S      2.437     0.01    9.797 6.994e-15     True     None     None
EMg_S     3.663     0.01    24.07 1.031e-14     True     None     None
ENa_S     1.966     0.01    8.459 5.475e-15     True     None     None
WCa_S     18.94        0   

Error of Observed Ca: RSSE(sim-obs): 11.044768
Error of Observed Mg: RSSE(sim-obs): 1.124561
Error of Observed Na: RSSE(sim-obs): 1.153668
Error of Observed K: RSSE(sim-obs): 0.238704
Error of Observed ECa: RSSE(sim-obs): 0.000004
Error of Observed EMg: RSSE(sim-obs): 0.000001
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 1.160285
Error of Observed SO4: RSSE(sim-obs): 7.077113


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S      17.5     0.01    73.16 6.885e-06     True     None     None
EK_S       4.32     0.01    13.45 1.824e-06     True     None     None
EMg_S     4.753     0.01    26.15 1.888e-06     True     None     None
ENa_S     1.386     0.01    7.359 5.862e-07     True     None     None
WCa_S     30.81        0   

Error of Observed Ca: RSSE(sim-obs): 2.679698
Error of Observed Mg: RSSE(sim-obs): 0.846645
Error of Observed Na: RSSE(sim-obs): 0.983651
Error of Observed K: RSSE(sim-obs): 1.230644
Error of Observed ECa: RSSE(sim-obs): 0.000007
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 1.304090
Error of Observed SO4: RSSE(sim-obs): 11.297071


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     32.77     0.01       90 1.064e-05     True     None     None
EK_S        4.1     0.01    12.23 1.488e-06     True     None     None
EMg_S     8.188     0.01    31.63 3.015e-06     True     None     None
ENa_S     4.089     0.01    11.74 1.624e-06     True     None     None
WCa_S     47.39        0   

Error of Observed Ca: RSSE(sim-obs): 24.222202
Error of Observed Mg: RSSE(sim-obs): 8.679636
Error of Observed Na: RSSE(sim-obs): 16.495741
Error of Observed K: RSSE(sim-obs): 2.283212
Error of Observed ECa: RSSE(sim-obs): 0.000865
Error of Observed EMg: RSSE(sim-obs): 0.000096
Error of Observed ENa: RSSE(sim-obs): 0.000127
Error of Observed EK: RSSE(sim-obs): 0.000030
Error of Observed NO3: RSSE(sim-obs): 2.934156
Error of Observed SO4: RSSE(sim-obs): 22.614038


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     29.14     0.01       90  0.02532     True     None     None
EK_S      4.261     0.01    12.59 0.004108     True     None     None
EMg_S     15.73     0.01       50  0.01437     True     None     None
ENa_S     5.324     0.01    14.09 0.006001     True     None     None
WCa_S     52.73        0     

Error of Observed Ca: RSSE(sim-obs): 10.356805
Error of Observed Mg: RSSE(sim-obs): 4.913659
Error of Observed Na: RSSE(sim-obs): 15.269849
Error of Observed K: RSSE(sim-obs): 0.306139
Error of Observed ECa: RSSE(sim-obs): 0.000000
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 6.971557
Error of Observed SO4: RSSE(sim-obs): 20.847429


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     23.68     0.01       90 1.23e-13     True     None     None
EK_S      3.574     0.01    11.43 1.97e-14     True     None     None
EMg_S     10.87     0.01    42.98 5.784e-14     True     None     None
ENa_S     4.008     0.01    11.82 2.369e-14     True     None     None
WCa_S     49.29        0   

Error of Observed Ca: RSSE(sim-obs): 3.244879
Error of Observed Mg: RSSE(sim-obs): 2.015712
Error of Observed Na: RSSE(sim-obs): 0.927890
Error of Observed K: RSSE(sim-obs): 0.179507
Error of Observed ECa: RSSE(sim-obs): 0.000436
Error of Observed EMg: RSSE(sim-obs): 0.000129
Error of Observed ENa: RSSE(sim-obs): 0.000019
Error of Observed EK: RSSE(sim-obs): 0.000009
Error of Observed NO3: RSSE(sim-obs): 16.870335
Error of Observed SO4: RSSE(sim-obs): 12.369555


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     21.85     0.01    83.41 0.0007479     True     None     None
EK_S      4.398     0.01    12.91 0.0001592     True     None     None
EMg_S     16.43     0.01       50 0.0005588     True     None     None
ENa_S      3.78     0.01    11.31 0.0001476     True     None     None
WCa_S     33.33        0  

Error of Observed Ca: RSSE(sim-obs): 3.017876
Error of Observed Mg: RSSE(sim-obs): 0.689037
Error of Observed Na: RSSE(sim-obs): 4.703910
Error of Observed K: RSSE(sim-obs): 0.200988
Error of Observed ECa: RSSE(sim-obs): 0.000000
Error of Observed EMg: RSSE(sim-obs): 0.000000
Error of Observed ENa: RSSE(sim-obs): 0.000000
Error of Observed EK: RSSE(sim-obs): 0.000000
Error of Observed NO3: RSSE(sim-obs): 4.146433
Error of Observed SO4: RSSE(sim-obs): 5.996538


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
NO3Sink_S      -100     -100        0     None     True     None     None
Name      Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S     5.374     0.01     33.8 1.176e-14     True     None     None
EK_S      3.165     0.01       11 6.779e-15     True     None     None
EMg_S     4.461     0.01     25.6 9.739e-15     True     None     None
ENa_S     1.473     0.01      7.8 3.23e-15     True     None     None
WCa_S     48.52        0    2